# Getting Corpus 
Searching google for the references and getting articles corresponding each reference

Making a change to the reference parsing algorithm : The current __get_hotwords__ method does not include numbers so an external method is created which is able to add numbers as they are relevant contextual information 
<br/>
        
    case in point, the following sentence 
    - About 1500 people were evacuated from dangerous areas of Luhansk region
    is translated as 
    - people dangerous areas luhansk region 
    
## Methodolgy

Since there are ~3000 references, we have decided to split the article dataset into 
l; oj;neipmvsd[lf 9w8uehf jwd][f
    

In [ ]:
import pandas as pd
from textblob import TextBlob
from newspaper import Article
from datetime import timedelta
import spacy
from collections import Counter
from string import punctuation

from googlesearch import search


import sys
sys.path.append('../')
import utility_functions


In [ ]:
nlp = spacy.load("en_core_web_md")

In [ ]:
reference = pd.read_csv('../data/processed/input-data.csv',parse_dates=['time_parsed'])

In [ ]:
len(reference)

## Testing on one reference 

In [ ]:
current_ref = reference.iloc[203]

In [ ]:
current_ref

In [ ]:
title = current_ref['title']
titleHotwords = utility_functions.get_hotwords(title, nlp)
modifiedHotwords = utility_functions.add_numerics(title,titleHotwords)
timeFrom = str(current_ref['time_parsed'].date()) # Date at the time of reporting
timeTo = str((current_ref['time_parsed'] + timedelta(days=2)).date())  # Date 2 days from the date of reporting 

In [ ]:
SEARCHTERM = ' '.join(modifiedHotwords) + ' after:' +timeFrom + ' before:' + timeTo
print(f'Searching for\n----- {SEARCHTERM}')

In [ ]:
# searches = 0 
# for searchResult in search(SEARCHTERM):
#     if 'liveuamap.com' in searchResult or 'wikipedia.org' in searchResult :
#         continue
        
#     searches = searches + 1 
#     print(searchResult)
    
    
#     article_parse = Article(searchResult)
#     article_parse.download()
#     article_parse.parse()
#     article_author = article_parse.authors
#     article_date = article_parse.publish_date
#     article_text = article_parse.text

#     if searches >10:
#         break

## Applying on all dataset

In [ ]:
current_ref

In [ ]:
# Batch 1 
start_1 = 0 
end_1 = 10
# end_1 = int(len(reference))

print(f'Getting Dataset From {start_1} to {end_1} ')

batch_1 = []

for index in range(start_1,end_1):
    current_ref = reference.iloc[index]
    title = current_ref['title']
    titleHotwords = utility_functions.get_hotwords(title, nlp)
    uid = current_ref['uid']
    modifiedHotwords = utility_functions.add_numerics(title,titleHotwords)
    timeFrom = str(current_ref['time_parsed'].date()) # Date at the time of reporting
    timeTo = str((current_ref['time_parsed'] + timedelta(days=2)).date())  # Date 2 days from the date of reporting 
    
    SEARCHTERM = ' '.join(modifiedHotwords) + ' after:' +timeFrom + ' before:' + timeTo
    print(f'Searching for\n----- {SEARCHTERM}')
    
    searches = 0 
    for searchResult in search(SEARCHTERM):
        if 'liveuamap.com' in searchResult or 'wikipedia.org' in searchResult :
            continue

        
        print(searchResult,end = ' ---')
        
        try:

            article_parse = Article(searchResult)
            article_parse.download()
            article_parse.parse()
            article_author = article_parse.authors
            article_date = article_parse.publish_date
            article_text = article_parse.text

            batch_1.append({
                'uid' : uid,
                'article_author' : article_author,
                'article_date' : article_date,
                'article_text' : article_text,
                'article_url' : searchResult
            })
            searches = searches + 1 
            print('Done')
        except Exception:
            print('Failed:Skipping')
        if searches >10:
            break

    